<a href="https://colab.research.google.com/github/ilyasoulk/big-gragas/blob/main/big_gragas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
import dateutil
import tarfile
import requests
import pandas as pd
import pandas.io.sql as sqlio

%matplotlib inline

In [2]:
stream = requests.get('https://www.lrde.epita.fr/~ricou/pybd/projet/boursorama2020.tgz', stream=True)

In [3]:
tarfile.open(fileobj=stream.raw, mode='r|gz').extractall('.') # try 'r:gz' if there is an error

In [4]:
os.listdir('2020')[:10]

['mercados 2020-10-09 17:41:01.529058',
 'amsterdam 2020-12-22 13:51:01.623672',
 'compA 2020-04-09 17:22:02.127051',
 'compB 2020-08-21 17:11:01.677186',
 'xetra 2020-06-24 09:31:01.414802',
 'compA 2020-07-29 16:21:02.252903',
 'compB 2020-02-07 14:42:01.795516',
 'bruxelle 2020-01-24 17:02:01.782035',
 'compB 2020-06-24 17:01:02.253402',
 'xetra 2020-09-11 17:11:01.648289']

In [37]:
import glob
from dateutil.parser import parse

file_paths_compA = glob.glob('2020/compA*')
fp_dict_compA = {parse(x.split('compA ')[1]):pd.read_pickle(x) for x in file_paths_compA}
compA = pd.concat(fp_dict_compA)

In [ ]:
# file_paths_compB = glob.glob('2020/compB *')
# fp_dict_compB = {parse(x.split('compB ')[1]):pd.read_pickle(x) for x in file_paths_compB}
# compB = pd.concat(fp_dict_compB)

In [ ]:
# file_paths_mercados = glob.glob('2020/mercados *')
# fp_dict_mercados = {parse(x.split('mercados ')[1]):pd.read_pickle(x) for x in file_paths_mercados}
# mercados = pd.concat(fp_dict_mercados)

In [ ]:
# file_paths_amsterdam = glob.glob('2020/amsterdam *')
# fp_dict_amsterdam = {parse(x.split('amsterdam ')[1]):pd.read_pickle(x) for x in file_paths_amsterdam}
# amsterdam = pd.concat(fp_dict_amsterdam)

In [ ]:
# file_paths_xetra = glob.glob('2020/xetra *')
# fp_dict_xetra = {parse(x.split('xetra ')[1]):pd.read_pickle(x) for x in file_paths_xetra}
# xetra = pd.concat(fp_dict_xetra)

In [ ]:
# file_paths_bruxelle = glob.glob('2020/bruxelle')
# fp_dict_bruxelle = {parse(x.split('bruxelle ')[1]):pd.read_pickle(x) for x in file_paths_bruxelle}
# bruxelle = pd.concat(fp_dict_bruxelle)

In [13]:
compA.dtypes

last      object
volume     int64
symbol    object
name      object
dtype: object

**Last column** is a *String* we need it to be a *Float*

In [14]:
last = compA['last']
last.head()

                            symbol 
2020-04-09 17:22:02.127051  1rPABBV    58.010(c)
                            1rPAC         28.570
                            1rPACNV    46.020(c)
                            1rPADP        91.800
                            1rPAF          5.214
Name: last, dtype: object

In [38]:
compA.rename_axis(['date', 'symbol'], inplace=True)
compA.drop(columns='symbol', inplace=True)
compA

last   volume              name
date                       symbol                                        
2020-04-09 17:22:02.127051 1rPABBV   58.010(c)        0            ABBVIE
                           1rPAC        28.570   505353             ACCOR
                           1rPACNV   46.020(c)        0             ACCOR
                           1rPADP       91.800    42036               ADP
                           1rPAF         5.214  1914174  AIR FRANCE - KLM
...                                        ...      ...               ...
2020-02-19 12:12:01.965109 1rPWLNNV   0.000(c)        0         WORLDLINE
                           1rPWLN       75.200   154320         WORLDLINE
                           1rPXPO      260.000       38     XPO LOGISTICS
                           1rPZC     24.830(c)        0       ZODIAC AERO
                           1rPZCNV   27.010(c)        0   ZODIAC AERO E17

[3976857 rows x 3 columns]

In [39]:
compA_reset = compA.reset_index()
compA_reset_duplicate = compA_reset.drop_duplicates()

KeyboardInterrupt: 

In [24]:
compA_reset_duplicate

,date,symbol,last,volume,name
0,2020-04-09 17:22:02.127051,1rPABBV,58.010(c),0,ABBVIE
1,2020-04-09 17:22:02.127051,1rPAC,28.570,505353,ACCOR
2,2020-04-09 17:22:02.127051,1rPACNV,46.020(c),0,ACCOR
3,2020-04-09 17:22:02.127051,1rPADP,91.800,42036,ADP
4,2020-04-09 17:22:02.127051,1rPAF,5.214,1914174,AIR FRANCE - KLM
...,...,...,...,...,...
3976852,2020-02-19 12:12:01.965109,1rPWLNNV,0.000(c),0,WORLDLINE
3976853,2020-02-19 12:12:01.965109,1rPWLN,75.200,154320,WORLDLINE
3976854,2020-02-19 12:12:01.965109,1rPXPO,260.000,38,XPO LOGISTICS
3976855,2020-02-19 12:12:01.965109,1rPZC,24.830(c),0,ZODIAC AERO


There are no duplicates in this dataframe

In [27]:
compA_na = compA.isna()
compA_na.query('last == True or volume == True or name == True')

,,last,volume,name
date,symbol,,,


No NaN cells

In [29]:
compA_null = compA.isnull()
compA_null.query('last == True or volume == True or name == True')

,,last,volume,name
date,symbol,,,


No Null cells as well

In [40]:
compA = compA.swaplevel(0, 1).sort_index()
compA

last  volume              name
symbol   date                                                          
1rAAF    2020-01-01 09:02:02.532411  9.612(c)       0  AIR FRANCE - KLM
         2020-01-01 09:12:01.997585  9.612(c)       0  AIR FRANCE - KLM
         2020-01-01 09:22:01.655711  9.612(c)       0  AIR FRANCE - KLM
         2020-01-01 09:32:02.484518  9.612(c)       0  AIR FRANCE - KLM
         2020-01-01 09:42:01.841351  9.612(c)       0  AIR FRANCE - KLM
...                                       ...     ...               ...
FF11_SGO 2020-12-31 17:11:01.942676    31.320       0      SAINT-GOBAIN
         2020-12-31 17:21:02.255744    31.320       0      SAINT-GOBAIN
         2020-12-31 17:31:01.759065    31.320       0      SAINT-GOBAIN
         2020-12-31 17:41:01.715188    31.320       0      SAINT-GOBAIN
         2020-12-31 17:51:02.225763    31.320       0      SAINT-GOBAIN

[3976857 rows x 3 columns]

In [32]:
day_off = ['2020-01-01', '2020-03-30', '2020-04-02', '2020-05-01', '2020-12-25', '2020-12-26']
for date in day_off:
  compA.drop(compA.loc[date].index, inplace=True) # This doesn't work, I'm unable to access a row by using only a part of a date

KeyError: '2020-01-01'

In [41]:
def format_last(x):
  try:
    return float(x)
  except:
    return float(x.split('(')[0].replace(' ', '')) # Split by ( to get rid of the (s) (c) then remove whitespace

compA['last'] = compA['last'].apply(format_last)
compA

last  volume              name
symbol   date                                                        
1rAAF    2020-01-01 09:02:02.532411   9.612       0  AIR FRANCE - KLM
         2020-01-01 09:12:01.997585   9.612       0  AIR FRANCE - KLM
         2020-01-01 09:22:01.655711   9.612       0  AIR FRANCE - KLM
         2020-01-01 09:32:02.484518   9.612       0  AIR FRANCE - KLM
         2020-01-01 09:42:01.841351   9.612       0  AIR FRANCE - KLM
...                                     ...     ...               ...
FF11_SGO 2020-12-31 17:11:01.942676  31.320       0      SAINT-GOBAIN
         2020-12-31 17:21:02.255744  31.320       0      SAINT-GOBAIN
         2020-12-31 17:31:01.759065  31.320       0      SAINT-GOBAIN
         2020-12-31 17:41:01.715188  31.320       0      SAINT-GOBAIN
         2020-12-31 17:51:02.225763  31.320       0      SAINT-GOBAIN

[3976857 rows x 3 columns]

In [44]:
compA[compA['name'] == 'AIR FRANCE - KLM']

last   volume              name
symbol date                                                        
1rAAF  2020-01-01 09:02:02.532411  9.612        0  AIR FRANCE - KLM
       2020-01-01 09:12:01.997585  9.612        0  AIR FRANCE - KLM
       2020-01-01 09:22:01.655711  9.612        0  AIR FRANCE - KLM
       2020-01-01 09:32:02.484518  9.612        0  AIR FRANCE - KLM
       2020-01-01 09:42:01.841351  9.612        0  AIR FRANCE - KLM
...                                  ...      ...               ...
1rPAF  2020-09-07 17:11:01.572668  3.815  2033396  AIR FRANCE - KLM
       2020-09-07 17:21:02.495603  3.814  2000902  AIR FRANCE - KLM
       2020-09-07 17:31:01.462402  3.812  2161202  AIR FRANCE - KLM
       2020-09-07 17:41:01.531471  3.807  2139571  AIR FRANCE - KLM
       2020-09-07 17:51:01.699988  3.812  2202657  AIR FRANCE - KLM

[20459 rows x 3 columns]